In [217]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

In [218]:
#defining utc datetime converter that catches null values
def utc_date(x):
    try:
        return(datetime.fromtimestamp(int(x['$date'])/1000).strftime('%H:%M:%S %m-%d-%Y'))
    except TypeError:
        return(None)

# Users

In [219]:
users = pd.read_json('users.json',lines=True)

In [220]:
users['_id'] = users['_id'].apply(lambda x: x['$oid'])
users['createdDate'] = users['createdDate'].apply(lambda x: utc_date(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: utc_date(x))

In [221]:
users.head()

_id  active          createdDate            lastLogin  \
0  5ff1e194b6a9d73a3a9f1052    True  10:24:04 01-03-2021  10:25:37 01-03-2021   
1  5ff1e194b6a9d73a3a9f1052    True  10:24:04 01-03-2021  10:25:37 01-03-2021   
2  5ff1e194b6a9d73a3a9f1052    True  10:24:04 01-03-2021  10:25:37 01-03-2021   
3  5ff1e1eacfcf6c399c274ae6    True  10:25:30 01-03-2021  10:25:30 01-03-2021   
4  5ff1e194b6a9d73a3a9f1052    True  10:24:04 01-03-2021  10:25:37 01-03-2021   

       role signUpSource state  
0  consumer        Email    WI  
1  consumer        Email    WI  
2  consumer        Email    WI  
3  consumer        Email    WI  
4  consumer        Email    WI

In [232]:
print("Number of rows:", len(users['_id']))
print("Number of null IDs:", sum(users['_id'].isna()))
print("Numer of unique IDs:", users['_id'].nunique())
print("Number of duplicate rows:", sum(users.duplicated()))

#there are about 283 duplicate rows

Number of rows: 495
Number of null IDs: 0
Numer of unique IDs: 212
Number of duplicate rows: 283


# Brands

In [223]:
brands = pd.read_json('brands.json',lines=True)

In [224]:
brands['_id'] = brands['_id'].apply(lambda x: x['$oid'])
brands_cpg = pd.json_normalize(brands['cpg'])
brands_cpg = brands_cpg.add_prefix('cpg.')

In [225]:
brands = pd.merge(brands, brands_cpg, left_index=True, right_index=True, how='outer')

In [226]:
brands.head()

_id       barcode        category      categoryCode  \
0  601ac115be37ce2ead437551  511111019862          Baking            BAKING   
1  601c5460be37ce2ead43755f  511111519928       Beverages         BEVERAGES   
2  601ac142be37ce2ead43755d  511111819905          Baking            BAKING   
3  601ac142be37ce2ead43755a  511111519874          Baking            BAKING   
4  601ac142be37ce2ead43755e  511111319917  Candy & Sweets  CANDY_AND_SWEETS   

                                                 cpg  \
0  {'$id': {'$oid': '601ac114be37ce2ead437550'}, ...   
1  {'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...   
2  {'$id': {'$oid': '601ac142be37ce2ead437559'}, ...   
3  {'$id': {'$oid': '601ac142be37ce2ead437559'}, ...   
4  {'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...   

                        name  topBrand                      brandCode  \
0  test brand @1612366101024       0.0                            NaN   
1                  Starbucks       0.0                      STARBUCKS   
2  test brand @1612366146176       0.0  TEST BRANDCODE @1612366146176   
3  test brand @1612366146051       0.0  TEST BRANDCODE @1612366146051   
4  test brand @1612366146827       0.0  TEST BRANDCODE @1612366146827   

  cpg.$ref              cpg.$id.$oid  
0     Cogs  601ac114be37ce2ead437550  
1     Cogs  5332f5fbe4b03c9a25efd0ba  
2     Cogs  601ac142be37ce2ead437559  
3     Cogs  601ac142be37ce2ead437559  
4     Cogs  5332fa12e4b03c9a25efd1e7

In [236]:
print("Number of rows:", len(brands['_id']))
print("Number of null IDs:", sum(brands['_id'].isna()))
print("Number of unique IDs:", brands['_id'].nunique())
print("Number of null barcodes:", sum(brands['barcode'].isna()))
print("Number of unique barcodes:", brands['barcode'].nunique())
print("Number of null categoryCode:", sum(brands['categoryCode'].isna()))
print("Number of unique categoryCode:", brands['categoryCode'].nunique())
print("Number of null brandCode:", sum(brands['brandCode'].isna()))
print("Number of unique brandCode:", brands['brandCode'].nunique())

#there are a few barcodes that are not unique with many of having null category codes and brand codes

Number of rows: 1167
Number of null IDs: 0
Number of unique IDs: 1167
Number of null barcodes: 0
Number of unique barcodes: 1160
Number of null categoryCode: 650
Number of unique categoryCode: 14
Number of null brandCode: 234
Number of unique brandCode: 897


# Receipts

In [244]:
receipts = pd.read_json('receipts.json',lines=True)

In [246]:
receipts_itemList = pd.json_normalize(receipts['rewardsReceiptItemList'])
receipts_itemList = receipts_itemList.add_prefix('rewardsReceiptItemList.')

In [249]:
receipts = pd.merge(receipts, receipts_itemList, left_index=True, right_index=True, how='outer')

In [251]:
receipts['_id'] = receipts['_id'].apply(lambda x: x['$oid'])
receipts['createDate'] = receipts['createDate'].apply(lambda x: utc_date(x))
receipts['dateScanned'] = receipts['dateScanned'].apply(lambda x: utc_date(x))
receipts['finishedDate'] = receipts['finishedDate'].apply(lambda x: utc_date(x))
receipts['modifyDate'] = receipts['modifyDate'].apply(lambda x: utc_date(x))
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].apply(lambda x: utc_date(x))
receipts['purchaseDate'] = receipts['purchaseDate'].apply(lambda x: utc_date(x))

In [253]:
receipts.head()

index                       _id  bonusPointsEarned  \
0      0  5ff1e1eb0a720f0523000575              500.0   
1      1  5ff1e1bb0a720f052300056b              150.0   
2      1  5ff1e1bb0a720f052300056b              150.0   
3      2  5ff1e1f10a720f052300057a                5.0   
4      3  5ff1e1ee0a7214ada100056f                5.0   

                             bonusPointsEarnedReason           createDate  \
0  Receipt number 2 completed, bonus point schedu...  10:25:31 01-03-2021   
1  Receipt number 5 completed, bonus point schedu...  10:24:43 01-03-2021   
2  Receipt number 5 completed, bonus point schedu...  10:24:43 01-03-2021   
3                         All-receipts receipt bonus  10:25:37 01-03-2021   
4                         All-receipts receipt bonus  10:25:34 01-03-2021   

           dateScanned         finishedDate           modifyDate  \
0  10:25:31 01-03-2021  10:25:31 01-03-2021  10:25:36 01-03-2021   
1  10:24:43 01-03-2021  10:24:43 01-03-2021  10:24:48 01-03-2021   
2  10:24:43 01-03-2021  10:24:43 01-03-2021  10:24:48 01-03-2021   
3  10:25:37 01-03-2021                 None  10:25:42 01-03-2021   
4  10:25:34 01-03-2021  10:25:34 01-03-2021  10:25:39 01-03-2021   

     pointsAwardedDate  pointsEarned  ... rewardsReceiptItemList.itemNumber  \
0  10:25:31 01-03-2021         500.0  ...                               NaN   
1  10:24:43 01-03-2021         150.0  ...                               NaN   
2  10:24:43 01-03-2021         150.0  ...                               NaN   
3                 None           5.0  ...                               NaN   
4  10:25:34 01-03-2021           5.0  ...                               NaN   

   rewardsReceiptItemList.originalMetaBriteQuantityPurchased  \
0                                                NaN           
1                                                NaN           
2                                                NaN           
3                                                NaN           
4                                                NaN           

  rewardsReceiptItemList.pointsEarned rewardsReceiptItemList.targetPrice  \
0                                 NaN                                NaN   
1                                 NaN                                NaN   
2                                 NaN                                NaN   
3                                 NaN                                NaN   
4                                 NaN                                NaN   

   rewardsReceiptItemList.competitiveProduct  \
0                                        NaN   
1                                        NaN   
2                                        NaN   
3                                        NaN   
4                                        NaN   

  rewardsReceiptItemList.originalFinalPrice  \
0                                       NaN   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   

  rewardsReceiptItemList.originalMetaBriteItemPrice  \
0                                               NaN   
1                                               NaN   
2                                               NaN   
3                                               NaN   
4                                               NaN   

  rewardsReceiptItemList.deleted rewardsReceiptItemList.priceAfterCoupon  \
0                            NaN                                     NaN   
1                            NaN                                     NaN   
2                            NaN                                     NaN   
3                            NaN                                     NaN   
4                            NaN                                     NaN   

  rewardsReceiptItemList.metabriteCampaignId  
0                                        NaN  
1                

In [262]:
print("Number of rows:", len(receipts['_id']))
print("Number of null IDs:", sum(receipts['_id'].isna()))
print("Number of unique IDs:", receipts['_id'].nunique())
#no instances of null or duplicate records

Number of rows: 7381
Number of null IDs: 0
Number of unique IDs: 1119


In [263]:
print(receipts.isnull().sum())

#there are lots of missing/null values, noticably in the nested json for receipt items, which raises concerns about data consistency
#in addition, lots of items in the receipt list are missing barcodes, which would make it difficult to link to other tables

index                                                           0
_id                                                             0
bonusPointsEarned                                            1401
bonusPointsEarnedReason                                      1401
createDate                                                      0
dateScanned                                                     0
finishedDate                                                 1411
modifyDate                                                      0
pointsAwardedDate                                            1301
pointsEarned                                                 1128
purchaseDate                                                  458
purchasedItemCount                                            484
rewardsReceiptItemList                                        440
rewardsReceiptStatus                                            0
totalSpent                                                    435
userId    